### Teste de hipotese

Esse notebook tem por objetivo detectar varios pontos e gerar um descritor otimo que seja resitente a variacoes de transformacoes afins e pequenas transformacoes projetivas, para isso temos:

-- BaseFeatures para extrair informacoes equivariantes (num_channels,dim_first,dim_second,dim_third).

-- SingularPoints lida com escala , e extrai as features consolidadas, em dim_third caracteristicas distintas, orientacao computadas além da lista de pontos.

-- Computa a funcao de perda entre os mapas de orientacao e feature e os pontos que colidiram

In [1]:
# !git clone -b main https://github.com/wagner1986/singular-points.git singular_points
# !pip install kornia e2cnn kornia_moons

# !pwd
# %cd /content/singular_points
# !pwd

# from google.colab import drive

# # Monta o Google Drive
# drive.mount('/content/drive')

In [2]:
import torch
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
from torch import nn
import numpy as np

In [3]:


def save_checkpoint(model, optimizer, epoch, loss, path):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
        'loss': loss,
        # Adicione outras informações que você deseja salvar, como hiperparâmetros, configurações, etc.
    }
    torch.save(checkpoint, path)
    
def load_checkpoint(model, optimizer, path):
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    # Outras informações que você salvou no dicionário de checkpoint podem ser acessadas aqui
    return model, optimizer, epoch, loss

In [4]:
from typing import List, Optional

import torch
import torch.nn.functional as F
from torch import nn
from typing_extensions import TypedDict

from kornia.core import Module, Tensor, concatenate
from kornia.filters import SpatialGradient
from kornia.geometry.transform import pyrdown
from kornia.utils.helpers import map_location_to_cpu

from kornia.feature.scale_space_detector import get_default_detector_config, MultiResolutionDetector,Detector_config


class KeyNet_conf(TypedDict):
    num_filters: int
    num_levels: int
    kernel_size: int
    Detector_conf: Detector_config


keynet_default_config: KeyNet_conf = {
    # Key.Net Model
    'num_filters': 8,
    'num_levels': 3,
    'kernel_size': 5,
    # Extraction Parameters
    'Detector_conf': {'nms_size': 5, 'pyramid_levels': 2, 'up_levels': 1, 'scale_factor_levels': 1.3, 's_mult': 20.0},
}


class _FeatureExtractor(Module):
    def __init__(self) -> None:
        super().__init__()

        self.hc_block = _HandcraftedBlock()
        self.lb_block = _LearnableBlock()

    def forward(self, x: Tensor) -> Tensor:
        x_hc = self.hc_block(x)
        x_lb = self.lb_block(x_hc)
        return x_lb


class _HandcraftedBlock(Module):
    def __init__(self) -> None:
        super().__init__()
        self.spatial_gradient = SpatialGradient('sobel', 1)

    def forward(self, x: Tensor) -> Tensor:
        sobel = self.spatial_gradient(x)
        dx, dy = sobel[:, :, 0, :, :], sobel[:, :, 1, :, :]

        sobel_dx = self.spatial_gradient(dx)
        dxx, dxy = sobel_dx[:, :, 0, :, :], sobel_dx[:, :, 1, :, :]

        sobel_dy = self.spatial_gradient(dy)
        dyy = sobel_dy[:, :, 1, :, :]

        hc_feats = concatenate([dx, dy, dx**2.0, dy**2.0, dx * dy, dxy, dxy**2.0, dxx, dyy, dxx * dyy], 1)

        return hc_feats


def _KeyNetConvBlock(
    feat_type_in,
    feat_type_out,
    r2_act,
    kernel_size: int = 5,
    stride: int = 1,
    padding: int = 2,
    dilation: int = 1,
) -> nn.Sequential:
    return enn.SequentialModule(
            enn.R2Conv(feat_type_in, feat_type_out, kernel_size=kernel_size, padding=padding, bias=False),
            enn.InnerBatchNorm(feat_type_out),
            enn.ReLU(feat_type_out, inplace=True),
        )


class _LearnableBlock(nn.Sequential):
    def __init__(self, in_channels: int = 10, out_channels: int = 8, group_size=8) -> None:
        super().__init__()
        r2_act = gspaces.Rot2dOnR2(N=group_size)

        feat_type_in = enn.FieldType(r2_act, in_channels * [r2_act.trivial_repr])
        self.in_type = feat_type_in
        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block0 = _KeyNetConvBlock(feat_type_in, feat_type_out, r2_act)

        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block1 = _KeyNetConvBlock(self.block0.out_type, feat_type_out, r2_act)

        feat_type_out = enn.FieldType(r2_act, out_channels * [r2_act.regular_repr])
        self.block2 = _KeyNetConvBlock(self.block1.out_type, feat_type_out, r2_act)
        self.gpool = enn.GroupPooling(self.block2.out_type)

    def forward(self, x: Tensor) -> Tensor:
        x = enn.GeometricTensor(x, self.in_type)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.gpool(x)
        return x.tensor

class KeyNet(Module):
    def __init__(self, pretrained: bool = False, keynet_conf: KeyNet_conf = keynet_default_config) -> None:
        super().__init__()

        num_filters = keynet_conf['num_filters']
        self.num_levels = keynet_conf['num_levels']
        kernel_size = keynet_conf['kernel_size']
        padding = kernel_size // 2

        self.feature_extractor = _FeatureExtractor()
        
        self.last_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=num_filters * self.num_levels, out_channels=1, kernel_size=kernel_size, padding=padding
            ),
            nn.ReLU(inplace=True),
        )
        if pretrained:
            KeyNet_URL ="./data/models/key_map_ep123.pth"
            load_checkpoint(self,None, KeyNet_URL)
            print("KeyNet loaded")


    def forward(self, x: Tensor) -> Tensor:
        shape_im = x.shape
        feats: List[Tensor] = [self.feature_extractor(x)]
        for i in range(1, self.num_levels):
            x = pyrdown(x, factor=1.2)
            feats_i = self.feature_extractor(x)
            feats_i = F.interpolate(feats_i, size=(shape_im[2], shape_im[3]), mode='bilinear')
            feats.append(feats_i)
        scores = self.last_conv(concatenate(feats, 1))
        return scores


class KeyNetDetector(MultiResolutionDetector):
    def __init__(
        self,
        pretrained: bool = False,
        num_features: int = 2048,
        keynet_conf: KeyNet_conf = keynet_default_config,
        ori_module: Optional[Module] = None,
        aff_module: Optional[Module] = None,
    ) -> None:
        model = KeyNet(pretrained, keynet_conf)
        super().__init__(model, num_features, keynet_conf['Detector_conf'], ori_module, aff_module)


In [5]:
import torch

def criar_mascara(size_batch,dimensao_janela, tamanho_borda):
    num_channels = 1
    mascara = torch.zeros((size_batch,num_channels, dimensao_janela, dimensao_janela), dtype=torch.uint8)
    mascara[..., tamanho_borda:-tamanho_borda, tamanho_borda:-tamanho_borda] = 1
    return mascara.to(torch.float32)

def my_similarity(a, b):
    # a_norm = torch.nn.functional.normalize(a.view(a.size(0), -1), dim=-1)
    # b_norm = torch.nn.functional.normalize(b.view(b.size(0), -1), dim=-1)
    # return torch.cdist(a_norm, b_norm, p=2)
    return torch.cdist(a.view(a.size(0), -1), b.view(b.size(0), -1), p=2)

# Crie métodos para calcular a perda
def loss_fn(map_anch, map_pos, margin=0.9):
    similarities = my_similarity(map_anch, map_pos)
    # Calcular a média da diagonal principal (âncoras vs. seus respectivos positivos)
    mean_diagonal = torch.mean(torch.diagonal(similarities))
    # Calcular a média dos outros elementos (âncoras vs. seus negativos correspondentes)
    mean_other = torch.mean(similarities[~torch.eye(similarities.shape[0], dtype=torch.bool)])
    losses = torch.relu(mean_diagonal - mean_other + margin)  # pos - neg + margin
    return losses,mean_diagonal,mean_other


def extract_feat_in_batch(model, batch_img):
    repo_features = torch.tensor([], dtype=torch.float).to(batch_img.device)
    for image in batch_img:
        feature = model(image[None])  # Adicione 'None' ou 'unsqueeze(0)' se necessário
        repo_features = torch.cat([repo_features, feature], dim=0)  # Coloque 'feature' dentro de uma lista []
    return repo_features

import gc
from tqdm import tqdm
def train_one_epoch(model, train_loader, loss_map, optimizer=None, device='cpu', transformations=None,is_training=True):
    model.train(is_training) # Set model to training mode
    total_loss = 0.
    desc="Train " if is_training else "Test "
    t = tqdm(train_loader, desc=desc)
    batch_i = 0
    loss_maps = 0.
    for batch_image, labels in t:
        batch_image = batch_image.to(device)
        mask = criar_mascara(batch_image.shape[0],batch_image.shape[-1],30).to(device)
        features_key_summary = extract_feat_in_batch(model,batch_image*mask)

        batch_t,mask_t,features_key_summary_t = transformations(batch_image,mask,features_key_summary)# transformar orientacoes e pontos
        features_key_summary_t2 = extract_feat_in_batch(model,batch_image*mask)# prever os pontos da imagem transformada


        loss,mean_diagonal,mean_other = loss_map(features_key_summary_t,features_key_summary_t2,margin = 70)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        t.set_description("{} Loss: {:.5f} Mean AP {:.5f} Mean AN {:.5f}".format(desc,loss,mean_diagonal,mean_other))
        del features_key_summary
        del batch_t, mask_t, features_key_summary_t
        del features_key_summary_t2
        gc.collect()
        torch.cuda.empty_cache()
        batch_i += 1
    return total_loss/batch_i



In [6]:
from teste_util import *
import teste_util as TS

# Fixar a semente do Torch para operações específicas
fixed_seed()

# leitura dos dados
trainloader,testloader =read_dataload_flower(120,'./data/datasets')
#gerar variacao de transformacoes pespectivas e fotometrica
iterator=iter(trainloader)
img,labels = next(iterator)
params_lists =AugmentationParamsGenerator(6,img.shape)

6072


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_keynet = KeyNet().to(device)
optimizer = optim.Adam(model_keynet.parameters(), lr=0.001, weight_decay=0.0001)
epoch_i=0
# model_keynet, optimizer, epoch_i, loss =load_checkpoint(model_keynet, optimizer,'./data/models/key_map_ep123.pth')
# print("epoch_i ",epoch_i,"loss ",loss)

/home/wagner/miniconda3/envs/singular-points/lib/python3.9/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647327249/work/aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)


In [8]:

from torch import optim
from torch.optim.lr_scheduler import ExponentialLR

transforms = kornia.augmentation.AugmentationSequential(
    kornia.augmentation.RandomAffine(degrees=360, translate=(0.2, 0.2), scale=(0.95, 1.05), shear=10,p=0.8),
    kornia.augmentation.RandomPerspective(0.2, p=0.7),
    kornia.augmentation.RandomBoxBlur((4,4),p=0.5),
    # kornia.augmentation.RandomEqualize(p=0.3),
    data_keys=["input","input","input"],
    same_on_batch=True,
    # random_apply=10,
)



gc.collect()
torch.cuda.empty_cache()
epochs=300
i_epoch = 0
loss = 0

scheduler = ExponentialLR(optimizer, gamma=0.75)

import torch

def train_with_early_stopping(model, trainloader, testloader, criterion_d, optimizer, scheduler, device, transformations, epochs=100, patience=20):
    best_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0

    for epoch in range(epoch_i,epochs):
        # Atualizar a taxa de aprendizado
        if (epoch % 5 == 0) and (epoch != 0):
            scheduler.step()
            

        running_loss = train_one_epoch(model, trainloader, loss_map=criterion_d,  optimizer=optimizer, device=device, transformations=transformations, is_training=True)

        with torch.no_grad():
            loss_test = train_one_epoch(model, testloader, loss_map=criterion_d,  optimizer=None, device=device, transformations=transformations, is_training=False)

        # Verificar se a perda melhorou
        if loss_test < best_loss:
            best_loss = loss_test
            epochs_without_improvement = 0
            best_model = model.state_dict()            
            save_checkpoint(model=model, epoch=epoch, optimizer=optimizer, loss=loss_test, path='./data/models/key_map_ep{}.pth'.format(epoch))
            print("salvou no colab")
        else:
            epochs_without_improvement += 1

        # Verificar a condição de parada
        if epochs_without_improvement == patience:
            print(f"No improvement in loss for {epochs_without_improvement} epochs. Training stopped.")
            break

        print(f"Epoch [{epoch}/{epochs}] - Running Loss: {running_loss:.4f}, Test Loss: {loss_test:.4f}, Initial LR: {optimizer.param_groups[0]['initial_lr']:.6f}, Current LR: {optimizer.param_groups[0]['lr']:.6f}, Epochs without Improvement: {epochs_without_improvement}")

    # Carregar a melhor configuração do modelo
    model.load_state_dict(best_model)
    print(f'Epoch: {epoch}, Best Loss: {best_loss:.4f}')

train_with_early_stopping(model_keynet.to(device), trainloader, testloader, loss_fn, optimizer, scheduler, device, transforms, epochs=epochs, patience=100)

Train  Loss: 62.78546 Mean AP 896.03326 Mean AN 903.24780: 100%|██████████| 17/17 [00:39<00:00,  2.33s/it]
Test  Loss: 0.00000 Mean AP 602.06274 Mean AN 706.93750: 100%|██████████| 102/102 [01:43<00:00,  1.02s/it] 


salvou no colab
Epoch [0/300] - Running Loss: 53.1452, Test Loss: 7.6109, Initial LR: 0.001000, Current LR: 0.001000, Epochs without Improvement: 0


Train  Loss: 45.87219 Mean AP 1136.40100 Mean AN 1160.52881: 100%|██████████| 17/17 [00:39<00:00,  2.32s/it]
Test  Loss: 0.03210 Mean AP 1149.11499 Mean AN 1219.08289:  82%|████████▏ | 84/102 [01:24<00:18,  1.01s/it] 


KeyboardInterrupt: 

### APOS treinamento a validacao

In [ ]:
detector = KeyNetDetector(pretrained=True, num_features=60, keynet_conf=keynet_default_config)
detector.to(device).eval()

descriptor = kornia.feature.SIFTDescriptor(TS.PS, rootsift=True).to(device)

image = torch.rand(1, 1, 128, 128).to(device)  # Imagem de exemplo 128x128 pixels em escala de cinza

# Faz a inferência com o modelo
with torch.no_grad():
    lafs, resps = detector(image)
    print(lafs.shape, resps.shape)

In [ ]:
import matplotlib.pyplot as plt
import kornia
from kornia_moons.viz import draw_LAF_matches

def detect_and_extract_features(image, detector, descriptor, PS):
    with torch.no_grad():
        lafs, resps = detector(image[None])
        patches = kornia.feature.extract_patches_from_pyramid(image[None], lafs, PS)
        B, N, CH, H, W = patches.size()
        # print('patches  ',patches.shape,B, N, CH, H, W,resps)
        descs = descriptor(patches.view(B * N, CH, H, W)).view(B, N, -1)

        return lafs, descs

def detect_extract_feat_in_batch(batch_img, detector, descriptor, PS):
    repo_lafs_desc = []
    with torch.no_grad():
        for image  in batch_img:
            try:
                lafs, descs = detect_and_extract_features(image, detector, descriptor, PS)
                repo_lafs_desc.append((lafs,descs))
            except RuntimeError as e:
                print("erro ao extrair features")

    return repo_lafs_desc

def matching_imagens(ref_img,batch_img, repo_lafs_desc,detector, descriptor):
    best_match_info = None
    best_match_count = 0
    best_match_index = None
    with torch.no_grad():
        # Detectar e extrair características da imagem de referência
        lafs_ref, descs_ref = detect_and_extract_features(ref_img, detector, descriptor, TS.PS)

        for i, (lafs_i, descs_i) in enumerate(repo_lafs_desc):
            # Detectar e extrair características da imagem atual do batch

            # matches = bidirectional_match(descs_ref[0], descs_i[0], threshold=0.85)
            scores, matches = kornia.feature.match_snn(descs_ref[0], descs_i[0], 0.85) # correspondencia dos descritories a uma distância de 0.9
            if matches.shape[0] >= 4:
                # Cálculo da homografia
                inliers_mask = compute_homography(lafs_ref, lafs_i, matches)

                # Check if this match is better than the previous best match
                if matches.shape[0] > best_match_count:
                    best_match_info = (lafs_ref[0][None].cpu(), lafs_i[0][None].cpu(), matches.cpu(),
                                       kornia.tensor_to_image(ref_img.cpu()), kornia.tensor_to_image(batch_img[i].cpu()),
                                       inliers_mask)
                    best_match_count = matches.shape[0]
                    best_match_index = i
        if best_match_info is not None and best_match_index==0:# TODO: Remove this condition best_match_index==0
            # Plot the best match

            draw_LAF_matches(
                *best_match_info,
                draw_dict={"inlier_color": (0.2, 1, 0.2), "tentative_color": (1, 1, 0.2, 0.3), "feature_color": None, "vertical": False},
            )
        # else:
        #     print("No matches found with enough inliers.")
    return best_match_index


In [ ]:
params_lists.aug_list.data_keys =["input"]
aug_list = params_lists.aug_list

acertos = 0
total = 0
from tqdm.notebook import tqdm
pbar =  tqdm(testloader)
for imgs_batch,labels_batch in pbar:# itera em todo dataset
    imgs_batch = imgs_batch.to(device)

    params_item = next(params_lists)
    timg_gray_t = aug_list(imgs_batch,params=params_item)
    plt.show()
    repo_lafs_desc= detect_extract_feat_in_batch(timg_gray_t,detector,descriptor,PS)

    for i,img_gray in enumerate(imgs_batch):# itera em cada batch

        match_index = matching_imagens(img_gray,timg_gray_t,repo_lafs_desc,detector, descriptor)

        total+=1
        if match_index == i:
            acertos += 1
        pbar.set_description(f"acertos/total: {acertos}/{total}  ")
print("acertos: ",acertos)

In [ ]:
with torch.no_grad():
    loss_test = train_one_epoch(detector.model, testloader, loss_map=loss_fn,  optimizer=None, device=device, transformations=transforms, is_training=False)
    print(f"Test loss: {loss_test:.4f}")